In [292]:
import soundfile as sf
from scipy.signal import lfilter
speech, sr = sf.read("/home/george/Downloads/eat.wav")

In [278]:
import numpy as np
import scipy.io
from olafilt import olafilt


def hz2erb(hz):
    # Convert normal frequency scale in hz to ERB-rate scale.
    # Units are number of Hz and number of ERBs.
    # ERB stands for Equivalent Rectangular Bandwidth.
    # Written by ZZ Jin, and adapted by DLW in Jan'07
    erb = 21.4 * np.log10(4.37e-3*hz+1)
    return erb


def erb2hz(erb):
    # Convert ERB-rate scale to normal frequency scale.
    # Units are number of ERBs and number of Hz.
    # ERB stands for Equivalent Rectangular Bandwidth.
    # Written by ZZ Jin, and adapted by DLW in Jan'07
    hz = (np.power(10, (erb / 21.4)) - 1) / 4.37e-3
    return hz


def loudness(freq):
    # Compute loudness level in Phons on the basis of equal-loudness functions.
    # It accounts a middle ear effect and is used for frequency-dependent gain adjustments.
    # This function uses linear interpolation of a lookup table to compute the loudness level, 
    # in phons, of a pure tone of frequency freq using the reference curve for sound 
    # pressure level dB. The equation is taken from section 4 of BS3383.
    # Written by ZZ Jin, and adapted by DLW in Jan'07
    dB = 60
    mat = scipy.io.loadmat("f_af_bf_cf.mat")
    if freq < 20 or freq > 12500:
        return
    i = 1

    ff = np.squeeze(mat["ff"])
    af = np.squeeze(mat["af"])
    bf = np.squeeze(mat["bf"])
    cf = np.squeeze(mat["cf"])

    while(ff[i] < freq):
        i += 1

    afy = af[i-1] + (freq - ff[i-1]) * \
        (af[i] - af[i-1]) / (ff[i] - ff[i-1])
    bfy = bf[i-1] + (freq - ff[i-1]) * \
        (bf[i] - bf[i-1]) / (ff[i] - ff[i-1])
    cfy = cf[i-1] + (freq - ff[i-1]) * \
        (cf[i] - cf[i-1]) / (ff[i] - ff[i-1])
    loud = 4.2 + afy * (dB-cfy) / (1 + bfy * (dB-cfy))
    return loud

def gammatone(sig_in, numChan=128, fRange=[80, 5000], fs=16000):
    # Produce an array of filtered responses from a Gammatone filterbank.
    # The first variable is required.
    # numChan: number of filter channels.
    # fRange: frequency range.
    # fs: sampling frequency.
    # Written by ZZ Jin, adapted by DLW in Jan'07 and JF Woodruff in Nov'08
    filterOrder = 4     # filter order
    gL = 2048   # gammatone filter length or 128 ms for 16 kHz sampling rate
    fRange = np.array(fRange)

    sigLength = len(sig_in)
    phase = np.zeros(numChan)  # initial phases
    erb_b = hz2erb(fRange)   # upper and lower bound of ERB29
    erb = np.linspace(erb_b[0], erb_b[1], numChan)  # ERB segment
    cf = erb2hz(erb)     # center frequency array indexed by channel
    b = 1.019 * 24.7 * (4.37 * cf / 1000 + 1)   # rate of decay or bandwidth

    # Generating gammatone impulse responses with middle-ear gain normalization
    gt = np.zeros((numChan, gL))    # Initialization
    tmp_t = np.arange(1, gL+1) / fs
    for i in range(numChan):
        gain = 10**((loudness(cf[i]) - 60) / 20) / 3 * \
            (2 * np.pi * b[i] / fs)**4    # loudness-based gain adjustments
        gt[i, :] = gain * fs**3 * tmp_t**(filterOrder-1) * \
            np.exp(-2*np.pi*b[i]*tmp_t) * \
            np.cos(2*np.pi*cf[i]*tmp_t+phase[i])
    #sig = np.reshape(sig_in, (sigLength, 1))   # convert input to column vector

    # gammatone filtering using FFTFILT
    r = np.zeros((numChan, sigLength))
    for i in range(numChan):
        tmp = olafilt(gt[i], sig_in)
        r[i][:] = tmp
    return r

In [143]:
g = gammatone(speech, 64, [50,8000], sr)

(64, 622667)
[[-3.55753838e-20 -8.17278408e-14 -7.78429501e-13 ...  1.06405504e-06
   1.20525082e-06  1.34468119e-06]
 [-1.52465931e-20 -2.42394756e-13 -2.30666800e-12 ... -4.72866489e-07
  -6.95650868e-07 -9.11692180e-07]
 [-7.11507676e-20 -5.77215464e-13 -5.48716505e-12 ... -1.04015112e-06
  -4.02146814e-07  2.37579743e-07]
 ...
 [-2.92734587e-18 -1.00172015e-10 -9.01244507e-10 ...  9.89426663e-05
   9.62513619e-05  9.13359602e-05]
 [ 5.20417043e-18 -1.22045367e-10 -1.08834843e-09 ...  5.67208225e-05
   3.44698676e-05  1.17441838e-05]
 [ 5.20417043e-18 -1.45929131e-10 -1.28829512e-09 ... -1.13281258e-04
  -1.26048443e-04 -1.31784731e-04]]


In [144]:
winLength = sr * 0.025
winShift = int(sr * 0.010)
numChan, sigLength = g.shape
increment = winLength / winShift
M = np.floor(sigLength / winShift).astype(int)
a = np.zeros((numChan, M))

In [145]:
ans = g[0, 0:3891*winShift]

In [146]:
ans.shape

(622560,)

In [147]:
def cochleagram(r, winShift, winLength=320):
    # Generate a cochleagram from responses of a Gammatone filterbank.
    # It gives the log energy of T-F units
    # The first variable is required.
    # winLength: window (frame) length in samples
    # Written by ZZ Jin, and adapted by DLW in Jan'07
    winShift = int(winShift)
    winLength = int(winLength)
    numChan, sigLength = r.shape

    increment = winLength / winShift
    M = np.floor(sigLength / winShift).astype(int)

    # calculate energy for each frame in each channel
    a = np.zeros((numChan, M))
    for m in range(M):
        for i in range(numChan):
            if m+1 < increment:       # shorter frame lengths for beginning frames
                a[i, m] = np.dot(r[i, 0:(m+1)*winShift], r[i, 0:(m+1)*winShift])
            else:
                startpoint = int((m + 1 - increment) * winShift)
                a[i, m] = np.dot(r[i, startpoint:startpoint+winLength], r[i, startpoint:startpoint+winLength])
    return a

In [270]:
def get_avg(m, v_span, h_span):
    # This function produces a smoothed version of cochleagram
    nr, nc = m.shape
    out = np.zeros((nr, nc))
    fil_size = (2 * v_span + 1) * (2 * h_span + 1)
    
    for i in range(nr):
        row_begin = 0
        row_end = nr
        col_begin = 0
        col_end = nc
        
        if (i - v_span) >= 0:
            row_begin = i - v_span
        if (i + v_span) <= nr-1:
            row_end = i + v_span + 1
        
        for j in range(nc):
            if (j - h_span) >= 0:
                col_begin = j - h_span
            if (j + h_span) <= nc-1:
                col_end = j + h_span + 1
            
            tmp = m[row_begin:row_end, col_begin:col_end]
            out[i,j] = np.sum(tmp) / fil_size
    return out

def deltas(x, w=9):
    nr, nc = x.shape

    # Define window shape
    hlen = np.floor(w/2)
    w = 2*hlen + 1
    win = np.arange(hlen, -hlen-1, -1)
    
    # pad data by repeating first and last columns
    xx = np.hstack((np.tile(x[:, 0], (int(hlen), 1)).T, x, np.tile(x[:, -1], (int(hlen), 1)).T))

    d = lfilter(win, 1, xx, 1)  # filter along dim 1 (rows)

    d = d[:, int(2*hlen):int(2*hlen)+nc]

    return d

In [228]:
coc = cochleagram(g, winLength = sr*0.025, winShift=sr*0.010)
coc[coc==0] = 10**-100
coc = np.log10(coc)

In [302]:
def MRCG_features(sig, sampFreq):
    # This function compute MRCG features
    beta = 1000 / np.sqrt(np.sum(sig**2) / len(sig))
    sig = sig * beta
    # sig = np.reshape(sig, (sig.shape[0], 1))
    g = gammatone(sig, 64, [50, 8000], sampFreq)    # Gammatone filterbank responses

    cochlea1 = cochleagram(g, winLength=sampFreq*0.025, winShift=sampFreq*0.010)
    #ochlea1[cochlea1 == 0] = 10**-100    # Avoid zero encountered in log10
    cochlea1 = np.log10(cochlea1)

    cochlea2 = cochleagram(g, winLength=sampFreq*0.200, winShift=sampFreq*0.010)
    #ochlea2[cochlea2 == 0] = 10**-100    # Avoid zero encountered in log10
    cochlea2 = np.log10(cochlea2)

    cochlea3 = get_avg(cochlea1, 5, 5)
    cochlea4 = get_avg(cochlea1, 11, 11)
    all_cochleas = np.vstack((cochlea1, cochlea2, cochlea3, cochlea4))
    delta = deltas(all_cochleas)
    ddelta = deltas(deltas(all_cochleas, 5), 5)
    
    mrcg_feat = np.vstack((all_cochleas, delta, ddelta))
    return mrcg_feat

In [303]:
mrcg = MRCG_features(speech, sr)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log10
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in log10
  


In [305]:
mrcg[:10, 280:290]

array([[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf]])

In [308]:
import scipy.io
mrcg_matlab = scipy.io.loadmat("~/mrcg.mat")
mrcg_matlab

FileNotFoundError: [Errno 2] No such file or directory: '~/mrcg.mat'